<a href="https://colab.research.google.com/github/ForeverPepe/TOVspII/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22TOVII2_1_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Теория обнаружения вторжений с применением искусственного интеллекта
# Практическая работа 2. Обнаружение атак на веб-приложения с применением обучения с подкреплением

## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import gym
from gym import spaces
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

## Чтение датафреймов

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/TOVII/2/NF-UQ-NIDS.csv')
data.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack,Dataset
0,149.171.126.0,62073,59.166.0.5,56082,6,0.0,9672,416,11,8,25,15,0,Benign,NF-UNSW-NB15
1,149.171.126.2,32284,59.166.0.5,1526,6,0.0,1776,104,6,2,25,0,0,Benign,NF-UNSW-NB15
2,149.171.126.0,21,59.166.0.1,21971,6,1.0,1842,1236,26,22,25,1111,0,Benign,NF-UNSW-NB15
3,59.166.0.1,23800,149.171.126.0,46893,6,0.0,528,8824,10,12,27,124,0,Benign,NF-UNSW-NB15
4,59.166.0.5,63062,149.171.126.2,21,6,1.0,1786,2340,32,34,25,1459,0,Benign,NF-UNSW-NB15


## Преобразование датаврейма

### Избавление от лишних атак

In [ ]:
attack_counts = data['Attack'].value_counts()
print(attack_counts)

Attack
Benign            9208048
DDoS               763285
Reconnaissance     482946
injection          468575
DoS                348962
Brute Force        291955
password           156299
xss                 99944
Infilteration       62072
Exploits            24736
scanning            21467
Fuzzers             19463
Backdoor            19029
Bot                 15683
Generic              5570
Analysis             1995
Theft                1909
Shellcode            1365
mitm                 1295
Worms                 153
ransomware            142
Name: count, dtype: int64


In [ ]:
data.drop(data[~data['Attack'].isin(['DDoS', 'injection', 'xss'])].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data.drop(columns=['Dataset', 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR'], inplace=True)
print(data['Attack'].value_counts())
data.head()

Attack
DDoS         763285
injection    468575
xss           99944
Name: count, dtype: int64


,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,60641,53,17,5.0,108,108,2,2,0,4,1,injection
1,60641,53,17,5.0,108,108,2,2,0,4,1,DDoS
2,38524,53,17,5.0,100,100,2,2,0,5,1,injection
3,38524,53,17,5.0,100,100,2,2,0,5,1,DDoS
4,42075,53,17,5.0,108,108,2,2,0,3,1,injection


### Преобразование категориальных данных в числовые

In [ ]:
encoder = LabelEncoder()
data['Attack'] = encoder.fit_transform(data['Attack'])
data.head()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,60641,53,17,5.0,108,108,2,2,0,4,1,1
1,60641,53,17,5.0,108,108,2,2,0,4,1,0
2,38524,53,17,5.0,100,100,2,2,0,5,1,1
3,38524,53,17,5.0,100,100,2,2,0,5,1,0
4,42075,53,17,5.0,108,108,2,2,0,3,1,1


## Разделение на выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['Attack']), data['Attack'], test_size=0.2, random_state=42)

## Создание среды RL

In [ ]:
class CyberSecurityEnv(gym.Env):
  def __init__(self, X, y):
    super(CyberSecurityEnv, self).__init__()
    self.X = X.values
    self.y = y.values
    self.current_index = 0
    self.action_space = spaces.Discrete(2)
    self.observation_space = spaces.Box(low=0, high=1, shape=(self.X.shape[1],), dtype=np.float32)

  def reset(self):
    self.current_index = 0
    return self.X[self.current_index]

  def step(self,action):
    correct_label = self.y[self.current_index]
    reward = 1 if action == correct_label else -1
    self.current_index += 1
    done = self.current_index >= len(self.X)
    return self.X[self.current_index % len(self.X)], reward, done, {}

env = CyberSecurityEnv(X_train, y_train)

## Разработка агента DQN

In [ ]:
def build_dqn_model(input_shape, action_size):
  model = Sequential([
      Dense(64, activation='relu', input_shape=(input_shape,)),
      Dense(64, activation='relu'),
      Dense(action_size, activation='linear')
  ])
  model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
  return model

dqn_model = build_dqn_model(X_train.shape[1], env.action_space.n)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Обучение агента

In [ ]:
def train_dqn(model, env, episodes=1000):
    gamma = 0.95
    epsilon = 1.0
    epsilon_min = 0.01
    epsilon_decay = 0.995
    memory = []
    batch_size = 32

    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        done = False

        while not done:
            if np.random.rand() <= epsilon:
                action = random.choice([0, 1])
            else:
                action = np.argmax(model.predict(np.array([state], dtype=np.float32))[0])

            next_state, reward, done, _ = env.step(action)
            memory.append((state, action, reward, next_state, done))
            total_reward += reward

            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                for s, a, r, s_next, d in minibatch:
                    target = r
                    if not d:
                        target += gamma * np.max(model.predict(np.array([s_next], dtype=np.float32))[0])
                    target_f = model.predict(np.array([s], dtype=np.float32))
                    target_f[0][a] = target
                    model.fit(np.array([s], dtype=np.float32), target_f, epochs=1, verbose=0)

            state = next_state

        epsilon = max(epsilon_min, epsilon * epsilon_decay)
        print(f"Episode {episode + 1}: Reward: {total_reward}")

## Оценка модели

In [ ]:
def evaluate_model(model, X, y):
  predictions = np.argmax(model.predict(X_test), axis=1)
  print(classification_report(y, predictions))

evaluate_model(dqn_model, X_test, y_test)

8324/8324 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step
              precision    recall  f1-score   support

           0       0.62      0.94      0.75    152873
           1       0.71      0.27      0.40     93517
           2       0.00      0.00      0.00     19971

    accuracy                           0.63    266361
   macro avg       0.44      0.40      0.38    266361
weighted avg       0.61      0.63      0.57    266361



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Сравнение с классическими методами

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Random Forest: ')
print(classification_report(y_test, y_pred_rf))

Random Forest: 
              precision    recall  f1-score   support

           0       0.91      0.90      0.90    152873
           1       0.64      0.67      0.66     93517
           2       0.02      0.02      0.02     19971

    accuracy                           0.75    266361
   macro avg       0.52      0.53      0.53    266361
weighted avg       0.75      0.75      0.75    266361

